In [3]:
import json
from pathlib import Path
import numpy as np

In [3]:
data_path = Path('../../data/samples.json')

In [5]:
with open(data_path, 'r') as file:
    for line in file:
        json_dict = json.loads(line)

        # Access and print all the keys in the dictionary
        keys_list = list(json_dict.keys())
        print(keys_list)

['bug_id', 'binary_severity', 'description', 'llama_tokenized_description', 'layer_id', 'hidden_state', 'text', 'tokenized']


In [16]:
with open(data_path, 'r') as file:
    for line in file:
        # Process each line as a separate JSON object
        json_dict = json.loads(line)

        # Access the elements in the dictionary for the current line
        bug_id = json_dict["bug_id"]
        # binary_severity = json_dict["binary_severity"]
        # description = json_dict["description"]
        # llama_tokenized_description = json_dict["llama_tokenized_description"]
        hidden_state = json_dict["hidden_state"]
        print('Length of hidden_state: ', len(hidden_state))
        hidden_state_np = np.array(hidden_state)
        # tokenized = json_dict["tokenized"]
        
        sum_aggregated_array = np.sum(hidden_state_np, axis=0)
        mean_aggregated_array = sum_aggregated_array / len(hidden_state_np)
        # do we have preference to conver in list everything?
        aggregated_list = mean_aggregated_array.tolist()
        print('aggregated_list lenght:', len(aggregated_list))
        print(aggregated_list)
        bug_severity = 0
        
        data = {
            "bug_id": bug_id,
            "binary_severity": bug_severity,
            "aggregated_list": aggregated_list,
        }
        with open('output_file.json', 'a') as outfile:
            json.dump(data, outfile)
            outfile.write('\n')

bug_id 125624
Length of hidden_state:  242
aggregated_list lenght: 5120
[0.0742886125548812, -0.5958119936225829, 0.16230843283913352, -0.07597380236160657, -0.33230770520927494, 0.27490890912773197, 0.534869863967265, -0.5459150499548794, -0.040682501044155155, 0.3848042448690115, -0.1666245805330513, -0.1323383987442521, 0.025349132285630408, 0.31240139322832594, -0.5119958357377485, 0.6350455165894564, 0.13912859829989346, -0.20493906194513495, -0.5733696173045261, 0.05569595935916113, -1.1104536608230968, -0.20557253813940632, 0.561870858688985, 0.20651189157785463, -0.4707879783693424, 0.30463780548946917, 0.44942124421931495, 0.702345958425979, -0.09949200212462875, -0.4856984004501469, 0.17759994632941634, 0.23202141060316858, -0.25785297992800876, -0.08470017161251099, -0.562827937859149, 0.05257504045470687, 0.08478175707099851, -0.3168996227674248, 0.42584285263187627, -0.13072749996973462, 0.2440443905917081, 0.20435143305250436, -1.9594381032896435, -0.17292681607333096, -0

# After the aggregation

In [1]:
import json

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import StratifiedShuffleSplit

from pathlib import Path

import numpy as np

In [2]:
file_path = Path('../../data/aggregation/output_file.json')

In [109]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.layer1(x)
        out = self.leaky_relu(out)
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

In [89]:
cont = 0

aggregated_lists = []
binary_severities = []

with open(file_path, 'r') as f:
    for line in f:
        line_data = line.strip(",\n")
        data_string = line_data.replace("'", '"')
        
        # Handling PosixPath object
        data_string = data_string.replace("PosixPath(", "")
        data_string = data_string.replace(")", "")
        
        aggregated_lists.append(eval(data_string)['aggregated_list'])
        binary_severities.append(eval(data_string)['binary_severity'])
        
        cont +=1
        if cont == 100:
            break

In [125]:
# Perform standard scaling on the features
# scaler = StandardScaler()
# aggregated_lists_scaled = scaler.fit_transform(aggregated_lists)

# Convert 'aggregated_lists' and 'binary_severities' to numpy arrays
X = np.array(aggregated_lists)
y = np.array(binary_severities)

# Use StratifiedShuffleSplit to split the data into training and testing sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Convert the numpy arrays back to lists
aggregated_lists_train = X_train.tolist()
aggregated_lists_test = X_test.tolist()
binary_severities_train = y_train.tolist()
binary_severities_test = y_test.tolist()

In [126]:
# Convert the lists to PyTorch tensors
aggregated_lists_tensor = torch.tensor(aggregated_lists_train, dtype=torch.float)
binary_severities_tensor = torch.tensor(binary_severities_train, dtype=torch.float)

# Create a TensorDataset from the tensors
dataset = TensorDataset(aggregated_lists_tensor, binary_severities_tensor)

In [127]:
# Define batch size and create a DataLoader
batch_size = 32  # Define your own batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [128]:
input_size = len(aggregated_lists[0])  # Assuming input size is the length of 'aggregated_list'
hidden_size = 64
output_size = 1

model = SimpleNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # lr = learning rate

In [133]:
# Training loop example
num_epochs = 100
total_samples = len(dataset)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.reshape([-1,1]))
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_samples//batch_size+1}], Loss: {loss.item():.4f}')

Epoch [1/100], Step [1/3], Loss: 0.5560
Epoch [1/100], Step [2/3], Loss: 0.5620
Epoch [1/100], Step [3/3], Loss: 0.5713
Epoch [11/100], Step [1/3], Loss: 0.4809
Epoch [11/100], Step [2/3], Loss: 0.5694
Epoch [11/100], Step [3/3], Loss: 0.5746
Epoch [21/100], Step [1/3], Loss: 0.5276
Epoch [21/100], Step [2/3], Loss: 0.4687
Epoch [21/100], Step [3/3], Loss: 0.6544
Epoch [31/100], Step [1/3], Loss: 0.5586
Epoch [31/100], Step [2/3], Loss: 0.5035
Epoch [31/100], Step [3/3], Loss: 0.5185
Epoch [41/100], Step [1/3], Loss: 0.5541
Epoch [41/100], Step [2/3], Loss: 0.4796
Epoch [41/100], Step [3/3], Loss: 0.5747
Epoch [51/100], Step [1/3], Loss: 0.5465
Epoch [51/100], Step [2/3], Loss: 0.5228
Epoch [51/100], Step [3/3], Loss: 0.5033
Epoch [61/100], Step [1/3], Loss: 0.4915
Epoch [61/100], Step [2/3], Loss: 0.5464
Epoch [61/100], Step [3/3], Loss: 0.5658
Epoch [71/100], Step [1/3], Loss: 0.5345
Epoch [71/100], Step [2/3], Loss: 0.5227
Epoch [71/100], Step [3/3], Loss: 0.5270
Epoch [81/100], Ste

In [134]:
aggregated_lists_test_tensor = torch.tensor(aggregated_lists_test, dtype=torch.float)
binary_severities_test_tensor = torch.tensor(binary_severities_test, dtype=torch.float)

test_tensor = TensorDataset(aggregated_lists_test_tensor, binary_severities_test_tensor)
test_data = DataLoader(test_tensor, batch_size=batch_size, shuffle=True)

In [135]:
# Set the model to evaluation mode
model.eval()

# Iterate through the test dataset
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_data:  # Iterate through your test dataset
        # labels = labels.reshape([-1,1])
        outputs = model(inputs)  # Forward pass
        print(outputs)
        predicted = (outputs > 0.5).float()  # Assuming binary classification with threshold 0.5
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate the accuracy
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

tensor([[9.1495e-02],
        [1.6375e-04],
        [5.1579e-01],
        [1.3801e-02],
        [7.5610e-05],
        [7.2795e-05],
        [9.0424e-01],
        [1.0321e-01],
        [9.8225e-01],
        [9.2407e-01],
        [3.7979e-02],
        [9.6929e-01],
        [9.8802e-01],
        [9.9998e-01],
        [9.9995e-01],
        [1.0000e+00],
        [8.8370e-06],
        [9.9252e-01],
        [9.4412e-04],
        [1.3474e-02]])
Test Accuracy: 10.0000


# Taking the confusion matrix

In [2]:
import sklearn.metrics as skMetr

In [1]:
import sys
sys.path.append('../llm/llama')

from main import plot_confusion

c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable


c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Import of import trl failed


In [10]:
file_path_pred = Path('../../data/aggregation/predictions.json')

true_list = []
pred_list = []
with open(file_path_pred, 'r') as json_file:
    for i,(line) in enumerate(json_file):
        try:
            line_data = line.strip(",\n")
            true_list.extend(eval(line_data)['binary_severity'])
            pred_list.extend(eval(line_data)['prediction'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")


In [12]:
len(pred_list)

4488

In [15]:
conf_matrix = skMetr.confusion_matrix(true_list, pred_list)
print(conf_matrix)
folder_path = Path('../../data/')

plot_confusion(conf_matrix, folder_path, unique_values = [0,1])


2023-10-25 15:14:53,479 - matplotlib.pyplot - DEBUG - Loaded backend agg version v2.2.


[[ 775  440]
 [ 537 2736]]


2023-10-25 15:14:53,744 - matplotlib.font_manager - DEBUG - findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
2023-10-25 15:14:53,752 - matplotlib.font_manager - DEBUG - findfont: score(FontEntry(fname='c:\\Users\\dessa\\anaconda3\\envs\\Master\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizOneSymBol.ttf', name='STIXSizeOneSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
2023-10-25 15:14:53,752 - matplotlib.font_manager - DEBUG - findfont: score(FontEntry(fname='c:\\Users\\dessa\\anaconda3\\envs\\Master\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSerif.ttf', name='DejaVu Serif', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
2023-10-25 15:14:53,752 - matplotlib.font_manager - DEBUG - findfont: score(FontEntry(fname='c:\\Users\\dessa\\anaconda3\\envs\\Master\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXS

In [4]:
import torch

labels = torch.tensor([0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1.])
predicted = torch.tensor([[0.],
                          [0.],
                          [0.],
                          [1.],
                          [0.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.],
                          [1.]])

# Adjust the shape of predicted to match the shape of labels
predicted = predicted.squeeze()

# Now labels and predicted have the same dimensions
print(labels)
print(predicted)

tensor([0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
        1., 1.])
tensor([0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])


In [5]:
from sklearn.metrics import accuracy_score, precision_score

# Convert the tensors to lists if they are not already lists
labels_list = labels.tolist() if not isinstance(labels, list) else labels
predicted_list = predicted if not isinstance(predicted, list) else predicted

# Calculate accuracy and precision
accuracy = accuracy_score(labels_list, predicted_list)
precision = precision_score(labels_list, predicted_list)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")

Accuracy: 0.75
Precision: 0.75
